In [332]:
r"""
Metric Groups
"""
# ****************************************************************************
#  Copyright (C) 2022      Ryan Johnson <johnsor@grace.edu>
#
#  Distributed under the terms of the GNU General Public License (GPL)
#                  https://www.gnu.org/licenses/
# *****************************************************************************
import sage
import itertools
from sage.rings.integer_ring import IntegerRing, ZZ
from sage.rings.number_field.number_field import CyclotomicField, NumberField_cyclotomic
from sage.rings.finite_rings.integer_mod_ring import Zmod
from sage.matrix.matrix_space import MatrixSpace
from sage.matrix.constructor import matrix
from sage.matrix.special import block_diagonal_matrix
from sage.groups.additive_abelian.additive_abelian_group import *
from sage.categories.category import Category
from sage.categories.number_fields import NumberFields
from sage.categories.modules import Modules
from sage.sets.positive_integers import PositiveIntegers
from sage.structure.unique_representation import UniqueRepresentation
from sage.structure.element import is_Matrix
from sage.misc.misc_c import prod
from sage.arith.misc import factor, legendre_symbol
from sage.arith.functions import lcm
from sage.functions.other import sqrt
from sage.modules.free_module_element import vector

def good_prd_tuple(p,r,d):
    if p == 2:
        d_range = range(1,7)
    else:
        d_range = range(1,3)
    return all((p.is_prime(), r>0, d in d_range))

def MetricGroup(prd_tuples):
    r"""
    Construct a metric group given a list of tuples, each determining
    an irreducible metric group.

    INPUT:

    - ``prd_tuples`` (list of tuples in \ZZ^3): these tuples should each
      be of the form `(p,r,d)` where `p` is a prime, `r` is a positive
      integer, and `d` determines the type of irreduciblle metric group.
      If `p` is an odd prime, then `d=1` or `d=2`.  If `p=2`, then
      `d=1,2,3,4,5` or `6`.  These follow the types given in
      https://arxiv.org/abs/1405.7950

    - ``remember_generators`` (boolean): whether or not to fix a set of
      generators (corresponding to the given invariants, which need not be in
      Smith form).

    OUTPUT:

    The metric group where `G = \bigoplus_i X_i(p,q)`, for each tuple `(p,r,d)` where
    `X = A` if `d=1` and so on alphabetically.
    """
    #check conditions on input
    prd_tuples = [(ZZ(p), ZZ(r), ZZ(d)) for (p,r,d) in prd_tuples]
    assert all(good_prd_tuple(p,r,d) for (p,r,d) in prd_tuples)
    orders = [p**r for (p,r,d) in prd_tuples for i in range(int(d/5)+1)]
    n = cyclotomic_n(orders)
    G = AdditiveAbelianGroup(orders)
    Z = CyclotomicField(n)
    Q = []
    for tup in prd_tuples:
        p,r,d = tup
        if p == 2 and d == 1:
            Q.append(matrix(ZZ, [[n*p**(-r-1)]]))
        elif p == 2 and d == 2:
            Q.append(matrix(ZZ, [[-n*p**(-r-1)]]))
        elif p == 2 and d == 3:
            Q.append(matrix(ZZ, [[5*n*p**(-r-1)]]))
        elif p == 2 and d == 4:
            Q.append(matrix(ZZ, [[-5*n*p**(-r-1)]]))
        elif p == 2 and d == 5:
            Q.append(ZZ(n*p**(-r-1))*matrix(ZZ, [[0,1],[1,0]]))
        elif p == 2 and d == 6:
            Q.append(ZZ(n*p**(-r-1))*matrix(ZZ, [[2,1],[1,2]]))
        elif d == 1:
            v = (p**r + 1)/2 #v is 2^(-1) in Z/p^rZ
            Q.append(matrix(ZZ, [[ZZ(v*n*p**(-r))]]))
        elif d == 2:
            v = (p**r + 1)/2 #v is 2^(-1) in Z/p^rZ
            u = ZZ(Zmod(p).multiplicative_generator())
            Q.append(matrix(ZZ, [[ZZ(u*v*n*p**(-r))]]))
        else:
            print("Build error: ", p, r, d)
    return PreMetricGroup(G,block_diagonal_matrix(Q, subdivide=False),Z)

#Finite Pre-Metric Group Element
class PreMetricGroupElement(AdditiveAbelianGroupElement):
    def rho(self):
        G = self.parent()
        Q = G._quadratic_matrix
        z = G._zeta
        x = vector(self.lift())
        return z**(x*Q*x)
    
    def chi(self, other):
        return (self+other).quadratic()/self.quadratic()/other.quadratic()
    
    def rho_a(self, other):
        return self.quadratic()*self.bilinear(other)
    
    def quadratic(self):
        G = self.parent()
        Q = G._quadratic_matrix
        n = G._modulo_n
        R = Integers(n)
        x = vector(self.lift())
        return R(x*Q*x)
    
    def bilinear(self, other):
        return (self+other).quadratic_additive()-self.quadratic_additive()-other.quadratic_additive()

class PreMetricGroup(AdditiveAbelianGroup_fixed_gens):
    Element = PreMetricGroupElement
    
    def __init__(self, invariants_or_group, coeffs_or_matrix, divisor_or_cyclotomic_field):        
        #Deal with the case when input is (*,*,CyclotomicField)
        if isinstance(divisor_or_cyclotomic_field, NumberField_cyclotomic):
            self._cyclotomic_field = divisor_or_cyclotomic_field
            self._modulo_n = divisor_or_cyclotomic_field._n()
        #Deal with the case when input is (*,*,positive_integer)
        elif divisor_or_cyclotomic_field in PositiveIntegers():
            self._cyclotomic_field = CyclotomicField(divisor_or_cyclotomic_field)
            self._modulo_n = ZZ(divisor_or_cyclotomic_field)
        else:
            raise TypeError("the third input must be a cyclotomic field or a positive integer")
        self._zeta = self._cyclotomic_field.zeta()
            
        #Deal with the case when input is (group,*,*)
        if isinstance(invariants_or_group, AdditiveAbelianGroup_class):
            cover = invariants_or_group.cover()
            rels = invariants_or_group.relations()
            gens = invariants_or_group.gens()
        #Deal with the case when input is (invariants,*,*)
        elif all([i in PositiveIntegers() for i in invariants_or_group]):
            cover, rels = cover_and_relations_from_invariants(invariants_or_group)
            gens = cover.gens()
        else:
            raise TypeError("the first input must be an additive abelian group or a list of invariants")
        
        #The underlying set is a finite additive abelian group
        AdditiveAbelianGroup_fixed_gens.__init__(self, cover, rels, gens)
        assert self.is_finite()
        
        #Deal with the case when inputs is (*,matrix,*)
        if is_Matrix(coeffs_or_matrix):
            assert self._is_homogeneous_quadratic(coeffs_or_matrix)
            self._quadratic_matrix = coeffs_or_matrix % self._modulo_n
        #Deal with the case when inputs is (*,coefficients,*)
        elif isinstance(coeffs_or_matrix, list):
            n = self.ngens()
            if len(coeffs_or_matrix) != n*(n+1)/2:
                raise TypeError("need to provide n(n+1)/2 coefficients for n generators")
            elif not all(i in ZZ for i in coeffs_or_matrix):
                raise TypeError("coefficients need to be integers")
            Q = matrix(ZZ, [[coeffs_or_matrix[j*n - j*(j-1)//2 + i - j] if i >=j
                             else coeffs_or_matrix[i*n - i*(i-1)//2 + j - i]
                             for i in range(n)] for j in range(n)])
            assert self._is_homogeneous_quadratic(Q)
            self._quadratic_matrix = Q % self._modulo_n
        else:
            raise TypeError("the second input must be an integer matrix or a" +
                            "list of the upper triangular coefficients")
       
    def _repr_(self):
        parts = ["Additive abelian group isomorphic to %s with quadratic form determined by \n"%self.short_name(),
                str(self._quadratic_matrix), "\n",
                "whose codomain is the %s"%self._cyclotomic_field]
        return "".join(parts)
      
    def sqrt(self, n):
        #This method exists because the sqrt method from Cyclotomic Fields did not always give
        #the positive square root in years past.
        z = self._zeta
        if not z.parent()(n).is_nth_power(2):
            raise ValueError("n has no square root in the cyclotomic field")
        n3 = z.multiplicative_order()
        sqrt_n = 1
        for fac in factor(n.squarefree_part()):
            if fac[0] == 2:
                sqrt_n = z**(n3/8) + z**(7*n3/8)
            else:
                p = fac[0]
                sqrt_n = sqrt_n * (1-z**(n3/4))/(1-z**(n3/4*legendre_symbol(-1,p))) * sum([legendre_symbol(i,p)*z**(n3*i/p) for i in range(p)])
        return sqrt_n * sqrt(n/n.squarefree_part())
    
    def gauss(self):
        n = self.cardinality()
        return self.sqrt(n)/n*sum([g.rho() for g in self])
    
    def _invariants_to_prime_powers():
        1+1
        #NOT DONE
    
    def _is_homogeneous_quadratic(self, Q):
        if Q.base_ring() != ZZ:
            raise TypeError("Q must be a matrix over the integers")
        
        #Check that Q maches the number of generators of G
        invs = [g.order() for g in self.gens()]
        w = len(invs)
        if Q.dimensions() != (w, w):
            raise TypeError("%s must match %s"%(Q, self.gens()))
        
        #Check that Q is homogeneous
        if not Q.is_symmetric():
            raise TypeError("Q must be a symmetric matrix")
            
        #Test that Q yields a bilinear form
        n = self._modulo_n
        non_bil = next(((i,j) for i in range(w) for j in range(w) if
                  not ZZ(n/invs[i]).divides(gcd(2*Q[i,j],n)) or
                  not ZZ(n/invs[j]).divides(gcd(2*Q[i,j],n))), None)
        if non_bil:
            raise TypeError("The orders of q_ij modulo %s and the orders of the"%n+
                            "i-th and j-th generators don't match for i=%s and j=%s"%non_bil)
        
        return True

        
def cyclotomic_n(invariants):
    """
    Given the invariants of group, return the smallest possible integer n
    such that we can define any quadratic form on the group to ZZ/nZZ,
    (1/n)ZZ/ZZ, or Cyclotomic(n).
    """
    n1 = lcm(invariants)
    n2 = prod(invariants).squarefree_part()
    #n1 is even, then output of quadratic form requires
    #the next highest power of 2
    if n1 % 2 == 0:
        n1 = n1*2
    #if n1 is odd, gauss sums will exist in QQ(i)
    else:
        n1 = n1*4
    #We need a cyclotomic field that contains sqrt(n)
    if n2 % 2 == 0:
        n2 = n2 * 4
    n3 = lcm(n1,n2)
    return n3

In [333]:
G = PreMetricGroup([2,4], [2,2,1], 8); G

Additive abelian group isomorphic to Z/2 + Z/4 with quadratic form determined by 
[2 2]
[2 1]
whose codomain is the Cyclotomic Field of order 8 and degree 4

In [334]:
MetricGroup([(2,2,5),(3,1,2)])

Additive abelian group isomorphic to Z/4 + Z/4 + Z/3 with quadratic form determined by 
[0 3 0]
[3 0 0]
[0 0 8]
whose codomain is the Cyclotomic Field of order 24 and degree 8

The code below is good at double checking the signature of a matric group, but it is slow.  It would be better to diagonalize the Q matrix and classify it's block matrices.

In [341]:
z = Z8.zeta()
s_additive = {z^j : j for j in range(8)}
s_additive[0] = oo
print(s_additive)

{1: 0, zeta8: 1, zeta8^2: 2, zeta8^3: 3, -1: 4, -zeta8: 5, -zeta8^2: 6, -zeta8^3: 7, 0: +Infinity}


In [345]:
G = MetricGroup([(2,2,1),(2,2,2),(3,2,2),(5,2,1)])
Z8 = CyclotomicField(8)
print(G)
n = G.order()
gens = G.gens()
ords = [g.order() for g in G.gens()]
primes = [p for (p,r) in factor(n)]
l = len(gens)
for p in primes:
    v_p = ZZ.valuation(p)
    ord_v = [v_p(o) for o in ords]
    G_p_slices = [[i*gens[j] for i in range(p^(max(ord_v[j],0)))] for j in range(l)]
    G_p = [sum(list(a)) for a in itertools.product(*G_p_slices)]
    #determine the power of p of tuple
    r = max(ord_v)
    sigma = []
    for k in range(r+1):
        H_len = prod([p^max(ord_v[i]-k,0) for i in range(l)])
        sqrt_H = G.sqrt(H_len)
        sigma += [Z8(sqrt_H/len(G_p)*sum([g.rho()^(p^k) for g in G_p]))]
    s_a = [s_additive[x] for x in sigma]
    print(p, " : ", sigma)
    print(p, " : ", s_a)

Additive abelian group isomorphic to Z/4 + Z/4 + Z/9 + Z/25 with quadratic form determined by 
[ 225    0    0    0]
[   0 1575    0    0]
[   0    0  200    0]
[   0    0    0  936]
whose codomain is the Cyclotomic Field of order 1800 and degree 480
2  :  [1, 1, 0]
2  :  [0, 0, +Infinity]
3  :  [1, zeta8^2, 1]
3  :  [0, 2, 0]
5  :  [1, -1, 1]
5  :  [0, 4, 0]


In [5]:
G = MetricGroup([(2,2,1),(2,2,2),(3,1,2),(5,2,2),(3,3,1)])
G._quadratic_matrix

[ 675    0    0    0    0]
[   0 4725    0    0    0]
[   0    0 1800    0    0]
[   0    0    0  216    0]
[   0    0    0    0 2800]

In [6]:
n = G.order()
gens = G.gens()
ords = [g.order() for g in G.gens()]
primes = [p for (p,r) in factor(n)]
print(primes, ords)

[2, 3, 5] [4, 4, 3, 25, 27]


In [7]:
g = G.0

In [8]:
g.order()

4

### Working on the diagonalization

In [508]:
ords = [2,2,2] #pick powers of an odd prime
p_r_ords = [factor(o) for o in ords]
if all(len(pr)==1 for pr in p_r_ords):
    p_r_ords = [o[0] for o in p_r_ords]
    p_r_ords_sorted = sorted(p_r_ords)
else:
    raise TypeError("group generators must have prime power order")
n = cyclotomic_n(ords)
coefs = [n/2,0,0,0,n/2,0]
A = PreMetricGroup(ords, coefs, n)
print(A)
gens = A.gens()
Q = A._quadratic_matrix
w = len(ords)
new_gens = list(gens)

Additive abelian group isomorphic to Z/2 + Z/2 + Z/2 with quadratic form determined by 
[4 0 0]
[0 0 4]
[0 4 0]
whose codomain is the Cyclotomic Field of order 8 and degree 4


In [509]:
Irreps = []
k=0
while k < w:
    print("k = ", k)
    p,r = p_r_ords_sorted[k] #this will give our (p,r,*,*) outputs in dictionary order for p and r
    print("p = ", p)
    u_p = ZZ(Integers(p).multiplicative_generator())
    print("u_p = ", u_p)
    #find the minimum v_p(q_ij) and call it rk
    rk = min(((q_ij/n).valuation(p) for (i,row_i) in enumerate(Q[k:])
             for (j,q_ij) in enumerate(row_i[k:]) if p_r_ords[i+k]==(p,r) or
             p_r_ords[j+k] == (p,r)), default=0)
    print("rk = ", rk)
    
    #If we find only zeros
    if rk >= 0:
        Irreps += [(p,r,1,r)]
        k+=1
        continue
    
    #does a diagonal element realize the minimum p-valuation?
    #if so, remember it's index
    i = next((i for (i,q_ii) in enumerate(Q.diagonal()[k:])
              if (q_ii/n).valuation(p)==rk and p_r_ords[i+k]==(p,r)), None)
        
    
    if p != 2:
        #if no diagonal element realizs the minimum p-valuation
        if i == None:
            #find the indices of an element that realizes the minimum p-valuation
            i,j = next((i,j) for (i,row_i) in enumerate(Q[k:])
                       for (j,q_ij) in enumerate(row_i[k:]) if(q_ij/n).valuation(p)==rk
                      and (p_r_ords[i+k]==(p,r) or p_r_ords[j+k]==(p,r)))
            #conjugating Q with S will make the i-th diagonal entry
            #realize the minimum p-valuation
            S = elementary_matrix(ZZ, w, row1=i+k, row2=j+k, scale = 1)
            S_inv = elementary_matrix(ZZ, w, row1=i+k, row2=j+k, scale = -1)
            print("----%s-%s-to-diagonal"%(i+k,j+k))
            print(Q);print(S)
            Q = S*Q*S.transpose() % n
            print(Q)
            new_gens = [A(h.lift()*S_inv) for h in new_gens]
            print(new_gens)
            print([g.quadratic() for g in new_gens])

        #move a diagonal element with minimum p-valuation to k,k.
        if i>0:
            S = elementary_matrix(ZZ, w, row1=k, row2=i+k)
            print("----%s-%s-to-%s-%s"%(i+k,i+k,k,k))
            print(Q);print(S)
            Q = S*Q*S.transpose() % n
            print(Q)
            new_gens = [A(h.lift()*S) for h in new_gens]
            print(new_gens)
            print([g.quadratic() for g in new_gens])
            p_r_ords[k], p_r_ords[k+i] = p_r_ords[k+i], p_r_ords[k]

        #scale the k,k entry to be 1 or u_p
        pr = ZZ(p**(-rk))
        print("pr = ", pr)
        R = Integers(pr)
        x = Q[k,k]*pr//n
        x_inv = R(x).inverse_of_unit()
        u_p_inv = R(u_p).inverse_of_unit()
        print("x = ", x, " and x_inv = ", x_inv)
        r = new_gens[k].order().valuation(p) #after reorganizing, our first generator has order p^r
        if R(x).is_square():
            a = R(x).square_root()
            Irreps += [(p,r,1,r+rk)]
        else:
            a = R(u_p_inv*x).square_root()
            Irreps += [(p,r,2,r+rk)]
        k+=1
        print("a = ", a)
        a_inv = a.inverse_of_unit()
        if a != R(1):
            S = elementary_matrix(ZZ, w, row1=k, scale = ZZ(a_inv))
            S_inv = elementary_matrix(ZZ, w, row1=k, scale = ZZ(a))
            print("----scale-row-&-col-%s-by-%s"%(k,a_inv))
            print(Q);print(S)
            Q = S*Q*S.transpose() % n
            print(Q)
            new_gens = [A(h.lift()*S_inv) for h in new_gens]
            print([g.quadratic() for g in new_gens])

        #sweep out
        x = Q[k,k]*pr//n
        x_inv = R(x).inverse_of_unit()
        for j in range(k+1,w):
            y = ZZ(x_inv)*Q[k,j]*pr//n
            if y != 0:
                S = elementary_matrix(ZZ, w, row1=j, row2=k, scale = -y)
                S_inv = elementary_matrix(ZZ, w, row1=j, row2=k, scale = y)
                print("----sweep out-the-%s-%s-entry"%(k,j))
                print("x = ", x, " x_inv = ", x_inv, " y = ", y)
                print(Q);print(S)
                Q = S*Q*S.transpose() % n
                print(Q)
                new_gens = [A(h.lift()*S_inv) for h in new_gens]
                print(new_gens)
                print([g.quadratic() for g in new_gens])
    else:
        #if no diagonal element realizs the minimum p-valuation
        if i == None:
            #find the indices of an element that realizes the minimum p-valuation
            i,j = next((i,j) for (i,row_i) in enumerate(Q[k:])
                       for (j,q_ij) in enumerate(row_i[k:]) if(q_ij/n).valuation(p)==rk
                      and (p_r_ords[i+k]==(p,r) or p_r_ords[j+k]==(p,r)))
            #identify if type E or type F
            pr = ZZ(p**(-rk))
            print("pr = ", pr)
            x = Q[k+i,k+j]*pr/n
            y = Q[k+i,k+i]*pr/(2*n)
            z = Q[k+j,k+j]*pr/(2*n)
            print("x = ", x)
            print("y = ", y)
            print("z = ", z)
            if (y*z).is_even():
                Irreps += [p,r,5,r+rk]
            else:
                Irreps += [p,r,6,r+rk]
            k+=2 #it's a 2-by-2 block            
            
            #if this block is in the upper left, we're done
            if (i+k,j+k) == (k,k+1):
                continue
            
            #If not, first move it to the upper left
            S = elementary_matrix(ZZ, w, row1=i+k, row2=j+k)
            print("----%s-%s-to-%s-%s"%(i+k,j+k,k,k+1))
            print(Q);print(S)
            Q = S*Q*S.transpose() % n
            print(Q)
            new_gens = [A(h.lift()*S_inv) for h in new_gens]
            print(new_gens)
            print([g.quadratic() for g in new_gens])
            
            #then sweep out
            #On 1/27/2022 I stopped here! TODO!
            continue
            
        #move a diagonal element with minimum p-valuation to k,k.
        if i>0:
            S = elementary_matrix(ZZ, w, row1=k, row2=i+k)
            print("----%s-%s-to-%s-%s"%(i+k,i+k,k,k))
            print(Q);print(S)
            Q = S*Q*S.transpose() % n
            print(Q)
            new_gens = [A(h.lift()*S) for h in new_gens]
            print(new_gens)
            print([g.quadratic() for g in new_gens])
            p_r_ords[k], p_r_ords[k+i] = p_r_ords[k+i], p_r_ords[k]
            
        #scale the k,k entry to be 1, -1, 5, or -5
        pr = ZZ(p**(-rk))
        print("pr = ", pr)
        R = Integers(pr)
        x = Q[k,k]*pr//n
        x_inv = R(x).inverse_of_unit()
        print("x = ", x, " and x_inv = ", x_inv)
        r = new_gens[k].order().valuation(p) #after reorganizing, our first generator has order p^r
        if R(x).is_square():
            a = R(x).square_root()
            Irreps += [(p,r,1,r+rk)]
        elif R(-x).is_square():
            a = R(-x).square_root()
            Irreps += [(p,r,2,r+rk)]
        elif R(5*x).is_square():
            a = R(5*x).square_root()
            Irreps += [(p,r,3,r+rk)]
        else:
            a = R(-5*x).square_root()
            Irreps += [(p,r,4,r+rk)]
        k+=1
        print("a = ", a)
        a_inv = a.inverse_of_unit()
        if a != R(1):
            S = elementary_matrix(ZZ, w, row1=k, scale = ZZ(a_inv))
            S_inv = elementary_matrix(ZZ, w, row1=k, scale = ZZ(a))
            print("----scale-row-&-col-%s-by-%s"%(k,a_inv))
            print(Q);print(S)
            Q = S*Q*S.transpose() % n
            print(Q)
            new_gens = [A(h.lift()*S_inv) for h in new_gens]
            print([g.quadratic() for g in new_gens])
            
        #sweep out
        x = Q[k,k]*pr//n
        x_inv = R(x).inverse_of_unit()
        for j in range(k+1,w):
            y = ZZ(x_inv)*Q[k,j]*pr//n
            if y != 0:
                S = elementary_matrix(ZZ, w, row1=j, row2=k, scale = -y)
                S_inv = elementary_matrix(ZZ, w, row1=j, row2=k, scale = y)
                print("----sweep out-the-%s-%s-entry"%(k,j))
                print("x = ", x, " x_inv = ", x_inv, " y = ", y)
                print(Q);print(S)
                Q = S*Q*S.transpose() % n
                print(Q)
                new_gens = [A(h.lift()*S_inv) for h in new_gens]
                print(new_gens)
                print([g.quadratic() for g in new_gens])
print(new_gens)
print(Irreps)
print(Q)

k =  0
p =  2
u_p =  1
rk =  -1
pr =  2
x =  1  and x_inv =  1
a =  1
k =  1
p =  2
u_p =  1
rk =  -1
pr =  2
x =  1
y =  0
z =  0
I stopped here
[(1, 0, 0), (0, 1, 0), (0, 0, 1)]
[(2, 1, 1, 0)]
[4 0 0]
[0 0 4]
[0 4 0]


In [ ]:
m = ords[-1]
fac = factor(m)
if len(fac) > 1:
    raise TypeError("only works when generators have prime power order")
else:
    p,r = fac[0]
print("p = ", p)
u_p = ZZ(Integers(p).multiplicative_generator())
rk = (Q[-1,-1]/n).valuation(p)
if rk >= 0:
    Irreps += [(p,r,1,r)]
else:
    pr = ZZ(p**(-rk))
    R = Integers(pr)
    print("pr = ", pr)
    x = Q[-1,-1]*pr//n
    u_p_inv = R(u_p).inverse_of_unit()
    print("x = ", x)
    if R(x).is_square():
        a = R(x).square_root()
        Irreps += [(p,r,1,r+rk)]
    else:
        a = R(u_p_inv*x).square_root()
        Irreps += [(p,r,2,r+rk)]

In [171]:
def min_val_to_diagonal(Q,n,k,S_all,S_inv_all):
    w = Q.nrows()
    r1 = min([v_p(x) for x in Q.list()])
    print("r1 = ", r1)
    i = next((i for (i,q_ii) in enumerate(Q.diagonal()[k:])
              if v_p(q_ii)==r1), None)
    if not i:
        i,j = next((i,j) for (i,row_i) in enumerate(Q[k:])
                   for (j,q_ij) in enumerate(row_i[k:]) if v_p(q_ij)==r1)
        S = elementary_matrix(ZZ, w, row1=i, row2=j, scale = 1)
        S_inv = elementary_matrix(ZZ, w, row1=i, row2=j, scale = -1)
        print("----")
        print(Q);print(S)
        Q = S*Q*S.transpose() % n
        print(Q)
        S_all = S*S_all
        S_inv_all = S_inv_all*S_inv
    return Q, S_all, S_inv_all, i

In [387]:
h = new_gens[0]

In [388]:
h.lift()

(6, 1, 0)

In [390]:
h._hermite_lift()

(0, 1, 0)

In [173]:
def min_val_to_k_k(Q,n,k,i,S_all,S_inv_all):
    w = Q.nrows()
    if i>k:
        S = elementary_matrix(ZZ, w, row1=k, row2=i)
        print("----")
        print(Q);print(S)
        Q = S*Q*S.transpose() % n
        print(Q)
        S_all = S*S_all
        S_inv_all = S_inv_all*S_inv
    return Q, S_all, S_inv_all

In [174]:
Q, S_all, S_inv_all = min_val_to_k_k(Q,n,0,i,S_all,S_inv_all)

----
[100 200 100]
[200 240 120]
[100 120 100]
[0 1 0]
[1 0 0]
[0 0 1]
[240 200 120]
[200 100 100]
[120 100 100]


In [ ]:
def scale_k_k(Q,n,k,S_all,S_inv_all):
    r1 = 
    pr = new_gens[0].order()//p**r1
    R = Integers(pr)
    x = Q[0,0]*pr//n
    u_p_inv = R(u_p).inverse_of_unit()
    print(x)
    if R(x).is_square():
        a = R(x).square_root().inverse_of_unit()
    else:
        a = R(u_p_inv*x).square_root().inverse_of_unit()
    print(a)
    if a != R(1):
        S = elementary_matrix(ZZ, l, row1=0, scale = ZZ(a))
        S_inv = elementary_matrix(ZZ, l, row1=0, scale = ZZ(a.inverse_of_unit()))
        print("----")
        print(Q);print(S)
        Q = S*Q*S.transpose() % n
        print(Q)
        new_gens = [A(h.lift()*S_inv) for h in new_gens]
        print(new_gens)
        for h in new_gens:
            x = vector(h.lift())
            print(x*Q*x % n,)

In [11]:
for k in range(A.ngens()):
    r1 = min([v_p(x) for x in Q.list()])
    new_Q, new_gens = min_val_to_upper_left(Q,new_gens,n,r1)

NameError: name 'min_val_to_upper_left' is not defined

check if one of the diagonal entrys valuation matches r1, and if so, return the index.  Else, find the off-diagon entry that matches.

### change this to a proper function, and zeros to k's

### archive: methods that work for the first row-column operations.

In [129]:
p = 7
ords = [p^2,p^3,p^3] #pick powers of an odd prime
n = cyclotomic_n(ords)
coefs = [n//p**1, n//p**1, n//p**1, n//p**1, n//p**2, n//p**1]
A = PreMetricGroup(ords, coefs, n)
print(A)
gens = A.gens()
Q = A._quadratic_matrix
v_p = ZZ.valuation(p)
u_p = ZZ(Integers(p).multiplicative_generator())
print("u_p = ", u_p)
l = len(ords)
new_gens = list(gens)

Additive abelian group isomorphic to Z/49 + Z/343 + Z/343 with quadratic form determined by 
[196 196 196]
[196 196  28]
[196  28 196]
whose codomain is the Cyclotomic Field of order 1372 and degree 588
u_p =  3


In [130]:
next((i for (i,q_ii) in enumerate(Q.diagonal()) if v_p(q_ii)==r1), None)

In [123]:
r1 = min([v_p(x) for x in Q.list()])
print("r1 = ", r1)
d_indices = [i for (i,q_ii) in enumerate(Q.diagonal()) if v_p(q_ii)==r1]
for h in new_gens:
    x = vector(h.lift())
    print(x*Q*x % n,)
if not d_indices:
    i,j = [(i,j) for (i,row_i) in enumerate(Q) for (j,q_ij) in enumerate(row_i) if v_p(q_ij)==r1][0]
    S = elementary_matrix(ZZ, l, row1=i, row2=j, scale = 1)
    S_inv = elementary_matrix(ZZ, l, row1=i, row2=j, scale = -1)
    print("----")
    print(Q);print(S)
    Q = S*Q*S.transpose() % n
    print(Q)
    new_gens = [A(h.lift()*S_inv) for h in new_gens]
    print(new_gens)
    for h in new_gens:
        x = vector(h.lift())
        print(x*Q*x % n,)
else:
    i,_ = d_indices[0]

r1 =  1
196
196
196
----
[196 196 196]
[196 196  28]
[196  28 196]
[1 0 0]
[0 1 1]
[0 0 1]
[196 392 196]
[392 448 224]
[196 224 196]
[(1, 0, 0), (0, 1, 342), (0, 0, 1)]
196
196
196


swap the diagonal entry whose valuation matches r1

In [106]:
if i!=0:
    S = elementary_matrix(ZZ, l, row1=0, row2=i)
    print("----")
    print(Q);print(S)
    Q = S*Q*S.transpose() % n
    print(Q)
    new_gens = [A(h.lift()*S) for h in new_gens]
    print(new_gens)
    for h in new_gens:
        x = vector(h.lift())
        print(x*Q*x % n,)

----
[196 392 196]
[392 448 224]
[196 224 196]
[0 1 0]
[1 0 0]
[0 0 1]
[448 392 224]
[392 196 196]
[224 196 196]
[(0, 1, 0), (1, 0, 342), (0, 0, 1)]
196
196
196


scale the 0,0 entry

In [107]:
pr = new_gens[0].order()//p**r1
R = Integers(pr)
x = Q[0,0]*pr//n
u_p_inv = R(u_p).inverse_of_unit()
print(x)
if R(x).is_square():
    a = R(x).square_root().inverse_of_unit()
else:
    a = R(u_p_inv*x).square_root().inverse_of_unit()
print(a)
if a != R(1):
    S = elementary_matrix(ZZ, l, row1=0, scale = ZZ(a))
    S_inv = elementary_matrix(ZZ, l, row1=0, scale = ZZ(a.inverse_of_unit()))
    print("----")
    print(Q);print(S)
    Q = S*Q*S.transpose() % n
    print(Q)
    new_gens = [A(h.lift()*S_inv) for h in new_gens]
    print(new_gens)
    for h in new_gens:
        x = vector(h.lift())
        print(x*Q*x % n,)

16
37
----
[448 392 224]
[392 196 196]
[224 196 196]
[37  0  0]
[ 0  1  0]
[ 0  0  1]
[ 28 784  56]
[784 196 196]
[ 56 196 196]
[(0, 1, 0), (4, 0, 342), (0, 0, 1)]
196
196
196


sweep out

In [108]:
x = Q[0,0]*pr//n
x_inv = R(x).inverse_of_unit()
y = ZZ(x_inv)*Q[0,1]*pr//n
if y != 0:
    S = elementary_matrix(ZZ, l, row1=1, row2=0, scale = -y)
    S_inv = elementary_matrix(ZZ, l, row1=1, row2=0, scale = y)
    print("----")
    print(Q);print(S)
    Q = S*Q*S.transpose() % n
    print(Q)
    new_gens = [A(h.lift()*S_inv) for h in new_gens]
    print(new_gens)
    for h in new_gens:
        x = vector(h.lift())
        print(x*Q*x % n,)

----
[ 28 784  56]
[784 196 196]
[ 56 196 196]
[  1   0   0]
[-28   1   0]
[  0   0   1]
[ 28   0  56]
[  0 196   0]
[ 56   0 196]
[(28, 1, 0), (4, 0, 342), (0, 0, 1)]
196
196
196


In [109]:
x = Q[0,0]*pr//n
x_inv = R(x).inverse_of_unit()
y = ZZ(x_inv)*Q[0,2]*pr//n
if y != 0:
    S = elementary_matrix(ZZ, l, row1=2, row2=0, scale = -y)
    S_inv = elementary_matrix(ZZ, l, row1=2, row2=0, scale = y)
    print("----")
    print(Q);print(S)
    Q = S*Q*S.transpose() % n
    print(Q)
    new_gens = [A(h.lift()*S_inv) for h in new_gens]
    print(new_gens)
    for h in new_gens:
        x = vector(h.lift())
        print(x*Q*x % n,)

----
[ 28   0  56]
[  0 196   0]
[ 56   0 196]
[ 1  0  0]
[ 0  1  0]
[-2  0  1]
[ 28   0   0]
[  0 196   0]
[  0   0  84]
[(28, 1, 0), (2, 0, 342), (2, 0, 1)]
196
196
196


In [37]:
n

100

In [38]:
52/4

13

In [50]:
(12*7*7) % 25

13

In [51]:
u_p_inv

13

In [52]:
(12*13) % 25

6

In [72]:
pr

125

In [89]:
pr

25